# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from sql_queries import insert_data_queries

#### Creating list of filepaths to process original event csv data files

In [19]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

all_filepath=[]

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

    file_path_list = glob.glob(os.path.join(root,'*'))
    all_filepath.append(file_path_list)
all_filepath = all_filepath[0][:]
all_filepath.sort()
print(all_filepath)

/home/workspace
['/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-19-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [18]:
song_all = []
for n, f in enumerate(all_filepath):
    
    df = pd.read_csv(f)
    
    song = df[['artist','firstName','gender','itemInSession','lastName','length',
               'level','location','sessionId','song','userId']]
    song = song.dropna(how='any')
    
    if n > 0:
        song_all = pd.concat([song_all, song], axis=0)
    else:
        song_all = song
        
song_all.to_csv("event_datafile_new.csv", encoding="utf-8", index=False)

#### Check how many rows will be in the csv file created

In [4]:
len(song_all)

6820

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_songs 
    WITH REPLICATION =
    {'class':'SimpleStrategy', 'replication_factor':1}""")

#### Set Keyspace

In [7]:
session.set_keyspace('music_songs')

## Created the queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Task1: Getting the artist and song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### FOR THIS QUERY ITEMINSESSION AND SESSIONID ARE CONSIDERED AS THE COMPOSITE KEY. BECAUSE THESE KEYS MAKE THE ROW UNIQUE AND BE ABLE TO KNOW THE SPECIFIC INFORMATION.

#### Create table

In [8]:
query = "CREATE TABLE IF NOT EXISTS song_per_session"
query = query +  """(sessionId int, iteminsession int, artist text, song text, 
length float, PRIMARY KEY((sessionId, iteminsession)))"""
session.execute(query)

#### Insert data

In [9]:
file = 'event_datafile_new.csv'
#read the csv by pandas
dfnew = pd.read_csv(file, encoding = 'utf8')
df1 = dfnew[['sessionId','itemInSession','artist','song','length']]

# insert records
for i, row in df1.iterrows():
    session.execute(insert_data_queries[0], (row.sessionId, row.itemInSession, row.artist, row.song, row.length))
    if i%1000==0:
        print("*")
print("Inserted to 1st table")

*
*
*
*
*
*
*
Inserted to 1st table


#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query_select = "SELECT sessionId, iteminsession, artist, song, length FROM song_per_session WHERE sessionId = 338 AND iteminsession = 4"

try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)
for r in rows:
    print("sessionId:{} | iteminsession:{} | artist:{} | song:{} | length: {}".format(r.sessionid, r.iteminsession,r.artist, r.song, r.length))
    print("*"*70)

sessionId:338 | iteminsession:4 | artist:Faithless | song:Music Matters (Mark Knight Dub) | length: 495.30731201171875
**********************************************************************


### Task2: Getting the name of artist, song (sorted by itemInSession) and user (first and last name)

##### FOR THIS QUERY USERID AND SESSIONID ARE CONSIDERED AS THE COMPOSITE KEY TO KNOW WHO LISTENED THE SONG AND WHAT SONG THE USER LISTNED DURING THE SESSION. ITEMINSESSIONID IS CONSIDERED AS CLUSTERING KEY IN ORDER TO SORT THE SONG'S ORDER IN ASCENDING.

#### Create table

In [11]:
query = "CREATE TABLE IF NOT EXISTS user_listened"
query = query +  "(userId int, sessionId int, iteminsession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), iteminsession))"
session.execute(query)

#### Insert data

In [12]:
file = 'event_datafile_new.csv'
df2 = dfnew[['userId' , 'sessionId', 'itemInSession', 'artist', 'song', 'firstName' , 'lastName']]
# insert records
for i, row in df2.iterrows():
    session.execute(insert_data_queries[1], (int(row.userId) , row.sessionId, row.itemInSession, row.artist, row.song, row.firstName , row.lastName))
    if i%1000==0:
        print("*")
print("Inserted to 2nd table")

*
*
*
*
*
*
*
Inserted to 2nd table


#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query_select2 = "SELECT artist, song, firstname, lastname FROM user_listened WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query_select2)
except Exception as e:
    print(e)
    
for row in rows:
    print("artist: {} | song: {} | firstname: {} | lastname: {}".format(row.artist, row.song, row.firstname , row.lastname))
    print("*"*70)

artist: Down To The Bone | song: Keep On Keepin' On | firstname: Sylvie | lastname: Cruz
**********************************************************************
artist: Three Drives | song: Greece 2000 | firstname: Sylvie | lastname: Cruz
**********************************************************************
artist: Sebastien Tellier | song: Kilometer | firstname: Sylvie | lastname: Cruz
**********************************************************************
artist: Lonnie Gordon | song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | firstname: Sylvie | lastname: Cruz
**********************************************************************


### Task3: Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### FOR THIS QUERY SONG AND USERID ARE CONSIDERED AS THE COMPOSITE KEY IN ORDER TO KNOW WHO LISTENED TO THE SPECIFIC SONG.

#### Create table

In [14]:
query = "CREATE TABLE IF NOT EXISTS wholistened_theSong"
query = query +  "(song text, userId int, firstName text, lastName text, PRIMARY KEY((song, userId)))"
session.execute(query)

#### Insert data

In [15]:
file = 'event_datafile_new.csv'

dfnew = pd.read_csv(file, encoding = 'utf8')

df3 = dfnew[['song', 'userId', 'firstName', 'lastName']]
df3.head(20)

for i, row in df3.iterrows():
    
    session.execute(insert_data_queries[2], (row.song, int(row.userId), row.firstName, row.lastName))

    if i%1000==0:
        print("*")

print("Inserted into 3rd table")

*
*
*
*
*
*
*
Inserted into 3rd table


#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
query_select3 = "SELECT  firstname , lastname \
FROM wholistened_theSong WHERE song = 'All Hands Against His Own' and userId = 80"
try:
    rows = session.execute(query_select3)
except Exception as e:
    print(e)
    
for row in rows:
    print("firstname: {} | lastname: {}".format( row.firstname , row.lastname))
    print("*"*70)

firstname: Tegan | lastname: Levine
**********************************************************************


### Drop the tables before closing out the sessions

In [ ]:
session.execute("drop table song_per_session")
session.execute("drop table user_listened")
session.execute("drop table wholistened_theSong")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()